In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/stat/rushing/season/2023/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [2]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [3]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [4]:
stats_columns = ['Position', 'Games Played', 'Rushing Attempts', 'Rushing Yards',
                 'Avg Yards per Rush Attempt', 'Long Rushing', '20+ Yard Rushes', 'Rushign Touchdowns',
                 'Rushing Yards per Game', 'Rushing Fumbles', 'Rushing Fumbles Lost', 'Rushing First Downs']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs
0,RB,16,272,"1,459",5.4,72,9,14,91.2,2,2,83
1,RB,17,280,"1,167",4.2,69,5,12,68.6,0,0,63
2,RB,12,228,"1,144",5.0,56,7,12,95.3,2,1,62
3,RB,17,237,"1,122",4.7,42,6,2,66.0,3,1,53
4,RB,16,229,"1,049",4.6,43,6,5,65.6,2,0,50
...,...,...,...,...,...,...,...,...,...,...,...,...
341,P,17,1,-11,-11.0,-11,0,0,-0.6,1,1,0
342,P,12,1,-11,-11.0,-11,0,0,-0.9,1,1,0
343,WR,10,1,-12,-12.0,-12,0,0,-1.2,0,0,0
344,QB,7,13,-13,-1.0,0,0,0,-1.9,0,0,0


In [5]:
total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats['Year'] = 2023
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Christian McCaffrey,SF,RB,16,272,"1,459",5.4,72,9,14,91.2,2,2,83,2023
1,2,Derrick Henry,TEN,RB,17,280,"1,167",4.2,69,5,12,68.6,0,0,63,2023
2,3,Kyren Williams,LAR,RB,12,228,"1,144",5.0,56,7,12,95.3,2,1,62,2023
3,4,James Cook,BUF,RB,17,237,"1,122",4.7,42,6,2,66.0,3,1,53,2023
4,5,D'Andre Swift,PHI,RB,16,229,"1,049",4.6,43,6,5,65.6,2,0,50,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,342,Johnny Hekker,CAR,P,17,1,-11,-11.0,-11,0,0,-0.6,1,1,0,2023
342,342,Ryan Stonehouse,TEN,P,12,1,-11,-11.0,-11,0,0,-0.9,1,1,0,2023
343,344,Justin Jefferson,MIN,WR,10,1,-12,-12.0,-12,0,0,-1.2,0,0,0,2023
344,345,Kyle Allen,BUF,QB,7,13,-13,-1.0,0,0,0,-1.9,0,0,0,2023


In [6]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Christian McCaffrey,SF,RB,16,272,"1,459",5.4,72,9,14,91.2,2,2,83,2023
1,2,Derrick Henry,TEN,RB,17,280,"1,167",4.2,69,5,12,68.6,0,0,63,2023
2,3,Kyren Williams,LAR,RB,12,228,"1,144",5.0,56,7,12,95.3,2,1,62,2023
3,4,James Cook,BUF,RB,17,237,"1,122",4.7,42,6,2,66.0,3,1,53,2023
4,5,D'Andre Swift,PHI,RB,16,229,"1,049",4.6,43,6,5,65.6,2,0,50,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,342,Johnny Hekker,CAR,P,17,1,-11,-11.0,-11,0,0,-0.6,1,1,0,2023
342,342,Ryan Stonehouse,TEN,P,12,1,-11,-11.0,-11,0,0,-0.9,1,1,0,2023
343,344,Justin Jefferson,MIN,WR,10,1,-12,-12.0,-12,0,0,-1.2,0,0,0,2023
344,345,Kyle Allen,BUF,QB,7,13,-13,-1.0,0,0,0,-1.9,0,0,0,2023


In [7]:
total_stats.to_csv("Rushing_2023.csv", index = False)
print("Data saved to Rushing_2023.csv")

Data saved to Rushing_2023.csv
